# 爬PTT地震文

* 需要的資訊：
    
    1. 發文時間
    2. 文章標題
    3. 文章內容
    4. 前100推推文內容
    5. 發文者ID
    6. 發文者IP位址
    7. 該文是否屬於地震文（dummy）
    8. 該文是否搶到地震爆文（dummy）
   
* 另外要寫的功能：

    1. 用IP位址反查地址資訊，回傳「縣市」（str）及經緯度（某種可以被餵進google map API的格式）
    2. 一天／一週當中PTT上站人數的pattern，輔以「當時發文者發文所處的時間」來控制error term

In [193]:
# -*- coding: utf-8 -*-

# 導入 模組(module) 
import requests 
# 導入 BeautifulSoup 模組(module)：解析HTML 語法工具
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

from multiprocessing import Pool #https://github.com/leVirve/CrawlerTutorial

In [194]:
#八卦版文章連結
url = "https://www.ptt.cc/bbs/Gossiping/index.html"
header_over18 = {"cookie":"over18=1"}
res = requests.get(url, headers = header_over18)
res

<Response [200]>

In [195]:
soup = BeautifulSoup(res.text, "html.parser")

# try to search keyword "地震"

In [196]:
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
resp = requests.get(search_endpoint_url, params={'q': '震'}, headers = header_over18)

In [197]:
soup = BeautifulSoup(resp.text, "html.parser")

In [198]:
# 找到全部地震文章的連結
links = soup.find_all("a href")

In [199]:
titles = soup.find_all("div", "title")

In [200]:
for title in titles:
    print(title.text, title.a["href"])
    link = "https://www.ptt.cc/" + title.a["href"]


Re: [問卦] 地震後愛莉莎莎沒事吧？
 /bbs/Gossiping/M.1591840928.A.FDC.html

Re: [問卦] 地震後愛莉莎莎沒事吧？
 /bbs/Gossiping/M.1591840584.A.E2D.html

[問卦] 地震後愛莉莎莎沒事吧？
 /bbs/Gossiping/M.1591839873.A.311.html

[問卦] 地震第一時間要做什麼才是正常人？
 /bbs/Gossiping/M.1591839848.A.0A8.html

[問卦] 地震時沒穿衣服怎麼辦？
 /bbs/Gossiping/M.1591839805.A.DF0.html

[爆卦] 地震！
 /bbs/Gossiping/M.1591839708.A.74D.html

[爆卦] 我愛你地震
 /bbs/Gossiping/M.1591839684.A.18F.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839641.A.DF7.html

[爆卦] 地震！
 /bbs/Gossiping/M.1591839618.A.C92.html

[問卦] 好震歐
 /bbs/Gossiping/M.1591839616.A.A62.html

地震
 /bbs/Gossiping/M.1591839606.A.C0F.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839605.A.CB1.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839603.A.59B.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839587.A.FB4.html

[爆卦] 震!!!
 /bbs/Gossiping/M.1591839582.A.8EB.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839573.A.5DF.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839572.A.872.html

[爆卦] 地震
 /bbs/Gossiping/M.1591839551.A.333.html

[新聞] 震驚！網紅破處卡關 爆羅志祥是「性幻
 /bbs/Gossiping/M.1591774559.A.D

In [201]:
article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
article_id

'1591772405'

In [202]:
# 進入搜尋頁面的第一篇文章
link = "https://www.ptt.cc/" + title.a["href"]

# 18禁
payload = {
    "from" : "/bbs/Gossiping/index.html",
    "yes"  : "yes"
}
rs = requests.session()
res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)

res_subpage = rs.get(link, verify = False)
soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

In [203]:
print(soup_subpage.text)






[新聞] 震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住 - 看板 Gossiping - 批踢踢實業坊
















批踢踢實業坊
›
看板 Gossiping
關於我們
聯絡資訊




返回看板




作者Tiara5566 (霸凌天團)看板Gossiping標題[新聞] 震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住時間Wed Jun 10 15:00:02 2020
自由

震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住

〔記者邱奕欽／台北報導〕歐陽娜娜將於下周一（15日）滿20歲，在生日前更爆出喜訊，
宣布正式加盟索尼音樂，並且推出首張創作EP《NANA I》，「很開心加入這個大家庭，有
許多很喜歡的師兄師姐，包含莫文蔚、鄧紫棋和周興哲等，有時會在節目上碰到，希望以
後可以和這個大家庭的歌手合作，一起玩出更多好的音樂給大家！」《NANA I》收錄三首
歐陽娜娜全創作的英文曲目，正式從古典誇界到流行音樂，她說：「從古典跨界流行需要
一個過程，有許多需要去學習和了解的部分，我覺得這兩者並不衝突。其實我從12歲開始
就有玩一些跨界，不管是中西合併的跨界或是時間上的跨界，這個部分有了一定學習和累
積。」歐陽娜娜5歲學鋼琴、6歲學習大提琴、11歲贏得台灣大提琴比賽冠軍、12歲成為最
年輕登上台灣最高規格音樂廳舉辦演奏會的演奏家，13歲考上聞名全球的美國柯蒂斯音樂
學院。此外，歐陽娜娜第一波宣傳照也令人驚艷，她穿著深V裸背白色禮服秀出好身材，
展現20歲半熟的女人味。

https://ent.ltn.com.tw/news/breakingnews/3191700

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 180.177.36.189 (臺灣)
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1591772405.A.3E6.html
噓 heinse: 我先 06/10 15:00
噓 woula: 中國人 06/10 15:00
噓 bamama56: 記得買榜 06/10 15:00
噓 Js1233: 賣台支那人必噓，其他就不用說了 06/10 15:00
→ Stylux: 可惡CD中 06/10 15:01
噓 peichuan: 現在的新聞標題真的超無聊,震個頭..... 06/

In [204]:
#一篇文章的 作者、標題、時間、看板
header = soup_subpage.find_all("span", "article-meta-value")
print(header)

[<span class="article-meta-value">Tiara5566 (霸凌天團)</span>, <span class="article-meta-value">Gossiping</span>, <span class="article-meta-value">[新聞] 震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住</span>, <span class="article-meta-value">Wed Jun 10 15:00:02 2020</span>]


In [205]:
print(header[0].text) #author
print(header[1].text) #board
print(header[2].text) #title
print(header[3].text) #date

Tiara5566 (霸凌天團)
Gossiping
[新聞] 震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住
Wed Jun 10 15:00:02 2020


In [206]:
main_text = soup_subpage.find(id = "main-container").text
print(main_text)


作者Tiara5566 (霸凌天團)看板Gossiping標題[新聞] 震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住時間Wed Jun 10 15:00:02 2020
自由

震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住

〔記者邱奕欽／台北報導〕歐陽娜娜將於下周一（15日）滿20歲，在生日前更爆出喜訊，
宣布正式加盟索尼音樂，並且推出首張創作EP《NANA I》，「很開心加入這個大家庭，有
許多很喜歡的師兄師姐，包含莫文蔚、鄧紫棋和周興哲等，有時會在節目上碰到，希望以
後可以和這個大家庭的歌手合作，一起玩出更多好的音樂給大家！」《NANA I》收錄三首
歐陽娜娜全創作的英文曲目，正式從古典誇界到流行音樂，她說：「從古典跨界流行需要
一個過程，有許多需要去學習和了解的部分，我覺得這兩者並不衝突。其實我從12歲開始
就有玩一些跨界，不管是中西合併的跨界或是時間上的跨界，這個部分有了一定學習和累
積。」歐陽娜娜5歲學鋼琴、6歲學習大提琴、11歲贏得台灣大提琴比賽冠軍、12歲成為最
年輕登上台灣最高規格音樂廳舉辦演奏會的演奏家，13歲考上聞名全球的美國柯蒂斯音樂
學院。此外，歐陽娜娜第一波宣傳照也令人驚艷，她穿著深V裸背白色禮服秀出好身材，
展現20歲半熟的女人味。

https://ent.ltn.com.tw/news/breakingnews/3191700

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 180.177.36.189 (臺灣)
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1591772405.A.3E6.html
噓 heinse: 我先 06/10 15:00
噓 woula: 中國人 06/10 15:00
噓 bamama56: 記得買榜 06/10 15:00
噓 Js1233: 賣台支那人必噓，其他就不用說了 06/10 15:00
→ Stylux: 可惡CD中 06/10 15:01
噓 peichuan: 現在的新聞標題真的超無聊,震個頭..... 06/10 15:01
推 conanhide: 前陣子都躲在台灣 06/10 15:01
噓 u87803170: (1/1) 06/10 15:01
噓 belucky: op   支那新聞 06/10 15:02
→ conanh

In [207]:
#推文
main_text.split(sep = "html")[1].split(sep = "\n")[1:]

['噓 heinse: 我先 06/10 15:00',
 '噓 woula: 中國人 06/10 15:00',
 '噓 bamama56: 記得買榜 06/10 15:00',
 '噓 Js1233: 賣台支那人必噓，其他就不用說了 06/10 15:00',
 '→ Stylux: 可惡CD中 06/10 15:01',
 '噓 peichuan: 現在的新聞標題真的超無聊,震個頭..... 06/10 15:01',
 '推 conanhide: 前陣子都躲在台灣 06/10 15:01',
 '噓 u87803170: (1/1) 06/10 15:01',
 '噓 belucky: op   支那新聞 06/10 15:02',
 '→ conanhide: 這種人真的愛中國？噁不噁心 06/10 15:02',
 '噓 kterry01: 江西娜 06/10 15:02',
 '噓 Lenney33: 來了 06/10 15:02',
 '噓 soarling: 幹你喵的是要發幾篇啦幹 06/10 15:02',
 '噓 icps: 真是中國 06/10 15:03',
 '→ makono: 記得打榜 06/10 15:03',
 '噓 k385476916: 滾回祖國再說 06/10 15:03',
 '噓 jiajia1: 棍 06/10 15:04',
 '→ mk5520: 一定買榜的 就是要她紅 06/10 15:04',
 '噓 amare1015: V殺小 小奶怪在那邊深V你老師個頭 06/10 15:05',
 '噓 okichan: 歐陽支那蒸蚌 06/10 15:06',
 '噓 charlietk3: 噁 06/10 15:06',
 '噓 FJU597660557: 贛．江西那 06/10 15:07',
 '噓 Hsiao810324: 先噓 06/10 15:07',
 '噓 sp038590: 摁摁 06/10 15:08',
 '噓 mete544: 嘔嘔嘔嘔嘔 06/10 15:08',
 '噓 lu3210153: 支那 06/10 15:08',
 '噓 jacky40383: 每日任務(1/1) 06/10 15:08',
 '噓 godbar: 滾 06/10 15:11',
 '噓 HsuKathy: 掰 

In [208]:
# 把整個內容切割透過 "-- " 切割成2個陣列
pre_text = main_text.split('--')[0]
    
# 把每段文字 根據 '\n' 切開
texts = pre_text.split('\n')
# 如果你爬多篇你會發現 
contents = texts[2:]
# 內容
content = '\n'.join(contents)

print('內容：'+content)

內容：自由

震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住

〔記者邱奕欽／台北報導〕歐陽娜娜將於下周一（15日）滿20歲，在生日前更爆出喜訊，
宣布正式加盟索尼音樂，並且推出首張創作EP《NANA I》，「很開心加入這個大家庭，有
許多很喜歡的師兄師姐，包含莫文蔚、鄧紫棋和周興哲等，有時會在節目上碰到，希望以
後可以和這個大家庭的歌手合作，一起玩出更多好的音樂給大家！」《NANA I》收錄三首
歐陽娜娜全創作的英文曲目，正式從古典誇界到流行音樂，她說：「從古典跨界流行需要
一個過程，有許多需要去學習和了解的部分，我覺得這兩者並不衝突。其實我從12歲開始
就有玩一些跨界，不管是中西合併的跨界或是時間上的跨界，這個部分有了一定學習和累
積。」歐陽娜娜5歲學鋼琴、6歲學習大提琴、11歲贏得台灣大提琴比賽冠軍、12歲成為最
年輕登上台灣最高規格音樂廳舉辦演奏會的演奏家，13歲考上聞名全球的美國柯蒂斯音樂
學院。此外，歐陽娜娜第一波宣傳照也令人驚艷，她穿著深V裸背白色禮服秀出好身材，
展現20歲半熟的女人味。

https://ent.ltn.com.tw/news/breakingnews/3191700




In [209]:
IP_address = main_text.split('來自:')[1].split("\n")[0]
IP_address

' 180.177.36.189 (臺灣)'

```
f"{article_id}" = {
    "Date" : ""
    "Title" : ""
    "Author" : ""
    "IP" : ""
    "Content" : 
}
```

In [210]:
f"{article_id}"

'1591772405'

In [211]:
exec_str = "M" + f"{article_id}"
exec_str

'M1591772405'

In [212]:
exec(exec_str+" = {}", globals())

In [213]:
type(M1590366890)

NameError: name 'M1590366890' is not defined

In [125]:
M1590366890[1] = "01"
M1590366890[2] = "02"

NameError: name 'M1590366890' is not defined

In [214]:
exec(f"{exec_str}", globals())

## 實作：把搜尋頁第一頁的文章都爬下來，存在一個dict內

In [215]:
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
#search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
#resp = requests.get(search_endpoint_url, params={'q': '地震'}, headers = header_over18)

#soup = BeautifulSoup(resp.text, "html.parser")

#忽略warning
import warnings
warnings.filterwarnings('ignore')

# 建立字典
earthquake_dict = {} # key是文章的link，value是另一本以文章的unique link為名的dict
# list內則有：發文時間、文章標題、作者ID、作者IP、文章內容
names = globals()

# 搜尋頁面換頁
# https://www.ptt.cc/bbs/Gossiping/search?page= 3 &q=%E5%9C%B0%E9%9C%87
# key換成index
index = 1
for page_count in range(1, 52+1):
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=%E5%9C%B0%E9%9C%87"
    search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=震"
    resp = requests.get(search_page_url, headers = header_over18)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 找到該搜尋頁面下 全部地震文章的連結
    titles = soup.find_all("div", "title") #找到每篇文章的標題及連結
    for title in titles:
        #print(title.text, title.a["href"])
        
        link = "https://www.ptt.cc/" + title.a["href"]
        article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
        article_id = index
        index+=1
        
        #為這篇文章建一個dictionary
        name_str = "M" + f"{article_id}"
        names["M_%s" %article_id] = {}
        earthquake_dict[name_str] = names["M_%s" %article_id]

        
        # 進入搜尋頁面的第一篇文章
        payload = {
            "from" : "/bbs/Gossiping/index.html",
            "yes"  : "yes"
        }
        rs = requests.session()
        res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)
        res_subpage = rs.get(link, verify = False)
        soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

        #一篇文章的 作者、標題、時間、看板
        header = soup_subpage.find_all("span", "article-meta-value")
        #print(header)
        #print(header[0].text) #author
        #print(header[1].text) #board
        #print(header[2].text) #title
        #print(header[3].text) #date
        
        try: #如果有「看板」的話
            names["M_%s" %article_id]["Author"] = header[0].text
            names["M_%s" %article_id]["Board"] = header[1].text
            names["M_%s" %article_id]["Title"] = header[2].text
            names["M_%s" %article_id]["Date"] = header[3].text
        except:
            try: #如果是回覆文章，沒有「看板」的話
                names["M_%s" %article_id]["Author"] = header[0].text
                names["M_%s" %article_id]["Title"] = header[1].text
                names["M_%s" %article_id]["Date"] = header[2].text
            except:
                pass

        # 內文
        try:
            main_text = soup_subpage.find(id = "main-container").text
        except:
            pass
        #print(main_text)
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = main_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = '\n'.join(contents)
        #print('內容：'+content)
        names["M_%s" %article_id]["Content"] = content

        
        #IP位址
        try:
            try:
                IP_address = main_text.split('來自:')[1].split("\n")[0]
                #print(IP_address)
                names["M_%s" %article_id]["Full IP"] = IP_address
            except:
                try:
                    author = header[0].text
                    authorID = author.split(sep = " (")[0]
                    #print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])
                    IP_address = main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:]
                    names["M_%s" %article_id]["Full IP"] = IP_address
                except:
                    pass
        except:
            names["M_%s" %article_id]["Full IP"] = "Not Found"
        
        #只有數字的IP
        try:
            IP_shorten = IP_address.split(sep = " ")[1]
            names["M_%s" %article_id]["IP"] = IP_shorten
        except:
            pass
        
        # 推文
        try:            
            push = main_text.split(sep = "html")[1].split(sep = "\n")[1:]
            names["M_%s" %article_id]["Push"] = push
        except:
            names["M_%s" %article_id]["Push"] = "No Push"
        
        time.sleep(0.1)

# 八卦版前一百頁

In [175]:
# 八卦版前一百頁
# https://www.ptt.cc/bbs/Gossiping/search?q=%E5%9C%B0%E9%9C%87 //查詢地震時的endpoint
#search_endpoint_url = 'https://www.ptt.cc/bbs/Gossiping/search'
# 把要查詢的關鍵字參數化
#resp = requests.get(search_endpoint_url, params={'q': '地震'}, headers = header_over18)

#soup = BeautifulSoup(resp.text, "html.parser")

#忽略warning
import warnings
warnings.filterwarnings('ignore')

# 建立字典
earthquake_dict = {} # key是文章的link，value是另一本以文章的unique link為名的dict
# list內則有：發文時間、文章標題、作者ID、作者IP、文章內容
names = globals()

# 搜尋頁面換頁
# https://www.ptt.cc/bbs/Gossiping/search?page= 3 &q=%E5%9C%B0%E9%9C%87
# key換成index
index = 1
for page_count in range(39284, 39184, -1):
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=%E5%9C%B0%E9%9C%87"
    #search_page_url = "https://www.ptt.cc/bbs/Gossiping/search?page=" + str(page_count) + "&q=震"
    search_page_url = "https://www.ptt.cc/bbs/Gossiping/index" + str(page_count) + ".html"
    resp = requests.get(search_page_url, headers = header_over18)
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 找到該搜尋頁面下 全部地震文章的連結
    titles = soup.find_all("div", "title") #找到每篇文章的標題及連結
    for title in titles:
        #print(title.text, title.a["href"])
        
        try:
            link = "https://www.ptt.cc/" + title.a["href"]
        except:
            pass
        #article_id = title.a["href"].split(sep = "/")[-1].split(sep = ".")[1]
        article_id = index
        index+=1
        
        #為這篇文章建一個dictionary
        name_str = "M" + f"{article_id}"
        names["M_%s" %article_id] = {}
        earthquake_dict[name_str] = names["M_%s" %article_id]

        
        # 進入搜尋頁面的第一篇文章
        payload = {
            "from" : "/bbs/Gossiping/index.html",
            "yes"  : "yes"
        }
        rs = requests.session()
        res_subpage = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)
        res_subpage = rs.get(link, verify = False)
        soup_subpage = BeautifulSoup(res_subpage.text, "html.parser")

        #一篇文章的 作者、標題、時間、看板
        header = soup_subpage.find_all("span", "article-meta-value")
        #print(header)
        #print(header[0].text) #author
        #print(header[1].text) #board
        #print(header[2].text) #title
        #print(header[3].text) #date
        
        try: #如果有「看板」的話
            names["M_%s" %article_id]["Author"] = header[0].text
            names["M_%s" %article_id]["Board"] = header[1].text
            names["M_%s" %article_id]["Title"] = header[2].text
            names["M_%s" %article_id]["Date"] = header[3].text
        except:
            try: #如果是回覆文章，沒有「看板」的話
                names["M_%s" %article_id]["Author"] = header[0].text
                names["M_%s" %article_id]["Title"] = header[1].text
                names["M_%s" %article_id]["Date"] = header[2].text
            except:
                pass

        # 內文
        try:
            main_text = soup_subpage.find(id = "main-container").text
        except:
            pass
        #print(main_text)
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = main_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = '\n'.join(contents)
        #print('內容：'+content)
        names["M_%s" %article_id]["Content"] = content

        
        #IP位址
        try:
            try:
                IP_address = main_text.split('來自:')[1].split("\n")[0]
                #print(IP_address)
                names["M_%s" %article_id]["Full IP"] = IP_address
            except:
                try:
                    author = header[0].text
                    authorID = author.split(sep = " (")[0]
                    #print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])
                    IP_address = main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:]
                    names["M_%s" %article_id]["Full IP"] = IP_address
                except:
                    pass
        except:
            names["M_%s" %article_id]["Full IP"] = "Not Found"
        
        #只有數字的IP
        try:
            IP_shorten = IP_address.split(sep = " ")[1]
            names["M_%s" %article_id]["IP"] = IP_shorten
        except:
            pass
        
        # 推文
        try:            
            push = main_text.split(sep = "html")[1].split(sep = "\n")[1:]
            names["M_%s" %article_id]["Push"] = push
        except:
            names["M_%s" %article_id]["Push"] = "No Push"
        
        time.sleep(0.1)

In [227]:
title

<div class="title">
<a href="/bbs/Gossiping/M.1579094206.A.089.html">[問卦] 剛剛的地震 座標位置是這樣嗎...</a>
</div>

In [228]:
header[0]

<span class="article-meta-value">lovemvkissy (還要繼續堅持)</span>

In [229]:
author = header[0].text
authorID = author.split(sep = " (")[0]
#print(main_text.split(sep = authorID)[2].split(sep = "),")[0].split()[0][1:])

In [230]:
authorID

'lovemvkissy'

In [231]:
page_count

52

In [232]:
len(earthquake_dict)

1040

In [233]:
earthquake_dict.keys()

dict_keys(['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 'M24', 'M25', 'M26', 'M27', 'M28', 'M29', 'M30', 'M31', 'M32', 'M33', 'M34', 'M35', 'M36', 'M37', 'M38', 'M39', 'M40', 'M41', 'M42', 'M43', 'M44', 'M45', 'M46', 'M47', 'M48', 'M49', 'M50', 'M51', 'M52', 'M53', 'M54', 'M55', 'M56', 'M57', 'M58', 'M59', 'M60', 'M61', 'M62', 'M63', 'M64', 'M65', 'M66', 'M67', 'M68', 'M69', 'M70', 'M71', 'M72', 'M73', 'M74', 'M75', 'M76', 'M77', 'M78', 'M79', 'M80', 'M81', 'M82', 'M83', 'M84', 'M85', 'M86', 'M87', 'M88', 'M89', 'M90', 'M91', 'M92', 'M93', 'M94', 'M95', 'M96', 'M97', 'M98', 'M99', 'M100', 'M101', 'M102', 'M103', 'M104', 'M105', 'M106', 'M107', 'M108', 'M109', 'M110', 'M111', 'M112', 'M113', 'M114', 'M115', 'M116', 'M117', 'M118', 'M119', 'M120', 'M121', 'M122', 'M123', 'M124', 'M125', 'M126', 'M127', 'M128', 'M129', 'M130', 'M131', 'M132', 'M133', 'M134', 'M135', 'M136', 'M137', '

In [245]:
earthquake_dict["M18"]

{'Author': 'B0Y0 (假柏濕)',
 'Board': 'Gossiping',
 'Title': '[爆卦] 地震',
 'Date': 'Thu Jun 11 09:39:09 2020',
 'Content': '到底ˋ有沒有\n\n\n第 030 號顯著有感地震報告\n發震時間：2020/06/11 09:38:26\n位置：北緯 24.5度，東經 122.4度\n即在宜蘭縣政府東偏南方 69.6 公里 ，位於臺灣東部海域\n地震深度：87公里\n芮氏規模：5.2\n\n各地震度級\n宜蘭縣地區最大震度 3級\n\n頭城 3級 宜蘭市 3級\n蘇澳 2級 南澳 2級\n澳花 2級 冬山 2級\n內城 2級 牛鬥 2級\n龜山島 1級 大同 1級 南山 1級\n\n\n花蓮縣地區最大震度 2級\n臺北市地區最大震度 2級\n南投縣地區最大震度 2級\n桃園市地區最大震度 2級\n嘉義縣地區最大震度 2級\n新北市地區最大震度 1級\n基隆市地區最大震度 1級\n新竹縣地區最大震度 1級\n臺中市地區最大震度 1級\n苗栗縣地區最大震度 1級\n臺東縣地區最大震度 1級\n彰化縣地區最大震度 1級\n雲林縣地區最大震度 1級\n\n',
 'Full IP': ' 114.45.248.37 (臺灣)',
 'IP': '114.45.248.37',
 'Push': ['推 sivhd: 有 06/11 09:39',
  '→ sincere77: 贏了 06/11 09:39',
  '推 sleepmilk: 怕 06/11 09:39',
  '推 CannedHamEgg: 有 06/11 09:39',
  '推 a30607212053: 有 06/11 09:39',
  '推 knoxdxd: 有 06/11 09:39',
  '推 sding: 你贏 06/11 09:39',
  '推 Ark727: 好大 06/11 09:39',
  '噓 corlos: yeah 06/11 09:39',
  '推 firelemon: 有 06/11 09:39',
  '推 ykap0969: 有 06/11 09:39',
  '推 william8403: you win 06/11 09:39',
  '→ bri

In [246]:
IP_shorten = IP_address.split(sep = " ")[1]
IP_shorten

'112.104.158.4'

# Save Dictionary

In [247]:
# Save
np.save("earthquake_dict_0611_6 兩小時後", earthquake_dict)

# Load
#read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()

In [226]:
earthquake_dict["M20"]

{'Author': 'Tiara5566 (霸凌天團)',
 'Board': 'Gossiping',
 'Title': '[新聞] 震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住',
 'Date': 'Wed Jun 10 15:00:02 2020',
 'Content': '自由\n\n震撼！歐陽娜娜爆喜訊 深V婚紗雪乳藏不住\n\n〔記者邱奕欽／台北報導〕歐陽娜娜將於下周一（15日）滿20歲，在生日前更爆出喜訊，\n宣布正式加盟索尼音樂，並且推出首張創作EP《NANA I》，「很開心加入這個大家庭，有\n許多很喜歡的師兄師姐，包含莫文蔚、鄧紫棋和周興哲等，有時會在節目上碰到，希望以\n後可以和這個大家庭的歌手合作，一起玩出更多好的音樂給大家！」《NANA I》收錄三首\n歐陽娜娜全創作的英文曲目，正式從古典誇界到流行音樂，她說：「從古典跨界流行需要\n一個過程，有許多需要去學習和了解的部分，我覺得這兩者並不衝突。其實我從12歲開始\n就有玩一些跨界，不管是中西合併的跨界或是時間上的跨界，這個部分有了一定學習和累\n積。」歐陽娜娜5歲學鋼琴、6歲學習大提琴、11歲贏得台灣大提琴比賽冠軍、12歲成為最\n年輕登上台灣最高規格音樂廳舉辦演奏會的演奏家，13歲考上聞名全球的美國柯蒂斯音樂\n學院。此外，歐陽娜娜第一波宣傳照也令人驚艷，她穿著深V裸背白色禮服秀出好身材，\n展現20歲半熟的女人味。\n\nhttps://ent.ltn.com.tw/news/breakingnews/3191700\n\n',
 'Full IP': ' 180.177.36.189 (臺灣)',
 'IP': '180.177.36.189',
 'Push': ['噓 heinse: 我先 06/10 15:00',
  '噓 woula: 中國人 06/10 15:00',
  '噓 bamama56: 記得買榜 06/10 15:00',
  '噓 Js1233: 賣台支那人必噓，其他就不用說了 06/10 15:00',
  '→ Stylux: 可惡CD中 06/10 15:01',
  '噓 peichuan: 現在的新聞標題真的超無聊,震個頭..... 06/10 15:01',
  '推 conanhide: 前陣子都躲在台灣 06/10 15:01',
  '